In [1]:
!pip install neuralforecast

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from ray import tune
import time
import logging
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoRNN, AutoLSTM, AutoGRU, AutoTCN, AutoDilatedRNN

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/lib/python3/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/lib/python3/dist-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [4]:
def load_data(file_path):
    df = (pd.read_csv(file_path)
          .rename(columns={"year_quarter": "ds", "GDPC1": "y"})
          .assign(unique_id=np.ones(len(pd.read_csv(file_path))),
                  ds=lambda df: pd.to_datetime(df['ds'])))
    columns_order = ["unique_id", "ds", "y"] + \
        [col for col in df.columns if col not in ["unique_id", "ds", "y"]]
    df['ds'] = df['ds'] - pd.Timedelta(days=1)
    return df[columns_order]


def separate_covariates(df, point_in_time):
    covariates = df.drop(columns=["unique_id", "ds", "y"])

    if not point_in_time:
        return df[covariates.columns], df[[]]

    mask = covariates.apply(
        lambda col: col.loc[col.index >= point_in_time - 1].isnull().any())

    past_covariates = df[mask.index[mask]]
    future_covariates = df[mask.index[~mask]]

    return past_covariates, future_covariates


def impute_missing_values_interpolate(data, method='linear'):
    imputed_data = data.copy()
    imputed_data.fillna(method='bfill', inplace=True)
    return imputed_data.interpolate(method=method)


### Different vintages ###


def forecast_vintage(vintage_file, horizon=4):
    results = {}

    df = load_data(vintage_file)

    target_df = df[["unique_id", "ds", "y"]]

    point_in_time = df.index[-2]

    past_covariates, future_covariates = separate_covariates(
        df, point_in_time)

    df_pc = impute_missing_values_interpolate(past_covariates)
    df_fc = impute_missing_values_interpolate(future_covariates)

    pcc_list = past_covariates.columns.tolist()
    fcc_list = future_covariates.columns.tolist()

    df = (target_df
          .merge(df_fc, left_index=True, right_index=True)
          .merge(df_pc, left_index=True, right_index=True)
          .iloc[:-1])

    futr_df = (target_df
               .merge(future_covariates, left_index=True, right_index=True)
               .drop(columns="y")
               .iloc[-1:])


    mlp_config = {
        "input_size": tune.choice([2, 4, 8, 12, 20]), # think about this tuning choice
        "hidden_size": tune.choice([256, 512, 1024]),
        "num_layers": tune.randint(2, 6),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "random_seed": tune.randint(1, 20),
        "hist_exog_list": tune.choice([pcc_list]),
        "futr_exog_list": tune.choice([fcc_list]),
        "max_steps": tune.choice([500, 1000]),
        "scaler_type": tune.choice(["robust"]),
    }

    # nbeats_config = {
    #     "input_size": tune.choice([2, 4, 12, 20]), # think about this tuning choice
    #     "learning_rate": tune.loguniform(1e-4, 1e-1),
    #     "scaler_type": tune.choice(["robust"]),
    #     "batch_size": tune.choice([32, 64, 128, 256]),
    #     "windows_batch_size": tune.choice([128, 256, 512, 1024]),
    #     #"dropout_prob_theta": tune.choice([0.1, 0.3, 0.5]),
    #     # "early_stop_patience_steps": tune.choice([-1, 2, 5]),
    #     #"hist_exog_list": tune.choice([pcc_list]),
    #     #"futr_exog_list": tune.choice([fcc_list]),
    #     "max_steps": tune.choice([500, 1000]),
    #     "random_seed": tune.randint(1, 20)
    # }

    nbeatsx_config = {
        "input_size": tune.choice([2, 4, 8, 12, 20]), # think about this tuning choice
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice(["robust"]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        #"dropout_prob_theta": tune.choice([0.1, 0.3, 0.5]),
        # "early_stop_patience_steps": tune.choice([-1, 2, 5]),
        "hist_exog_list": tune.choice([pcc_list]),
        "futr_exog_list": tune.choice([fcc_list]),
        "max_steps": tune.choice([500, 1000]),
        "random_seed": tune.randint(1, 20)
    }

    nhits_config = {
        "n_pool_kernel_size": tune.choice(
            [[2, 2, 1], 3 * [1], 3 * [2], 3 * [4], [8, 4, 1], [16, 8, 1]]
        ),
        "n_freq_downsample": tune.choice(
            [
                [168, 24, 1],
                [24, 12, 1],
                [180, 60, 1],
                [60, 8, 1],
                [40, 20, 1],
                [1, 1, 1],
            ]
        ),
        "input_size": tune.choice([2, 4, 8, 12, 20]), # think about this tuning choice
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice(["robust"]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        #"dropout_prob_theta": tune.choice([0.1, 0.3, 0.5]),
        #"early_stop_patience_steps": tune.choice([-1, 2, 5]),
        "hist_exog_list": tune.choice([pcc_list]),
        "futr_exog_list": tune.choice([fcc_list]),
        "max_steps": tune.choice([500, 1000]),
        "random_seed": tune.randint(lower=1, upper=20),
    }

    # Define models and their configurations
    models = {  
    "AutoMLP": {"config": mlp_config},
    # "AutoNBEATS": {"config": nbeats_config},
    "AutoNBEATSx": {"config": nbeatsx_config},
    "AutoNHITS": {"config": nhits_config},
    }

    model_instances = []

    for model_name, kwargs in models.items():
        print(f"Running model: {model_name}")
        model_class = globals()[model_name]
        # instance = model_class(h=horizon, num_samples=1, search_alg=HyperOptSearch(), loss=MAE(), **kwargs) 
        instance = model_class(h=horizon, num_samples=30, **kwargs) 
        model_instances.append(instance)

    nf = NeuralForecast(models=model_instances, freq='Q')
    nf.fit(df=df)

    Y_hat_df = nf.predict(futr_df=futr_df)
    forecast_value = Y_hat_df.iloc[:, 1].values.tolist()

    results[vintage_file] = forecast_value

    Y_hat_df = Y_hat_df.reset_index()

    return Y_hat_df, results 

In [5]:
comparison = pd.DataFrame()
results = {}

vintage_files = [
    f'data/vintage_{year}_{month:02d}.csv'
    for year in range(2018, 2024)
    for month in range(1, 13)
    if not (
        (year == 2018 and month < 5) or
        (year == 2023 and month > 2)
    )
]

In [ ]:

total_vintages = len(vintage_files)

start_time_whole = time.time()

def write_to_csv(df, block_number):
    df.to_csv(f'results/mlp_results_{block_number}.csv', index=False)

block_size = 1
for i in range(0, len(vintage_files), block_size):
    block = vintage_files[i:i+block_size]
    for j, vintage_file in enumerate(block):
        print(f"Processing {vintage_file} ({j+1}/{block_size}) in block {i//block_size + 1}")
        vintage_comparison, vintage_results = forecast_vintage(vintage_file)

        vintage_file_name = os.path.basename(vintage_file)  
        vintage_file_name = os.path.splitext(vintage_file_name)[0] 
        vintage_comparison = vintage_comparison.assign(vintage_file = vintage_file_name)

        comparison = pd.concat([comparison, vintage_comparison], ignore_index=True)
        
        results.update(vintage_results)
    
    write_to_csv(comparison, i//block_size + 1)



end_time_whole = time.time()

time_diff = end_time_whole - start_time_whole
hours, remainder = divmod(time_diff, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"Time taken to run the code: {int(hours)} hour(s), {int(minutes)} minute(s), and {seconds:.2f} seconds")

(pid=701377) /home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
(pid=701377)   from pandas.core.computation.check import NUMEXPR_INSTALLED
(pid=701377) /usr/lib/python3/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/lib/python3/dist-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
(pid=701377)   warn(f"Failed to load image Python extension: {e}")
(train_tune pid=701377) Global seed set to 17
(train_tune pid=701377) --------------------------------------------------------------------------
(train_tune pid=701377) WARNING: No preset parameters were found for the device that Open MPI
(train_tune pid=701377) detected:
(train_tune pid=701377) 
(train_tune pid=701377)   Local host:            152-67-225-75
(tra

Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 161.99it/s, v_num=0, train_loss_step=0.337, train_loss_epoch=1.250]
Validation: 0it [00:00, ?it/s] 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 163.69it/s, v_num=0, train_loss_step=0.338, train_loss_epoch=3.850, valid_loss=0.150]
Validation: 0it [00:00, ?it/s] 
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 138.99it/s, v_num=0, train_loss_step=0.181, train_loss_epoch=0.200, valid_loss=0.153]
Validation: 0it [00:00, ?it/s] 
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 137.45it/s, v_num=0, train_loss_step=0.287, train_loss_epoch=0.468, valid_loss=0.155]
Validation: 0it [00:00, ?it/s] 
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 156.00it/s, v_num=0, train_loss_step=1.880, train_loss_epoch=0.177, valid_loss=0.237]
Validation: 0it [00:00, ?it/s] 
Epoch 599: 100%|██████████| 1/1 [00:00<00:00, 163.11it/s, v_num=0, train_loss_step=0.195, train_loss_epoch=0.151, valid_loss=0.151]
Validation: 0it [00:00, ?it/s] 
Epoch 699: 100%|██████████| 1/1 [00

(train_tune pid=701377) Global seed set to 18


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 140.61it/s, v_num=0, train_loss_step=0.385, train_loss_epoch=0.699]
Validation: 0it [00:00, ?it/s] 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 169.53it/s, v_num=0, train_loss_step=0.729, train_loss_epoch=0.648, valid_loss=0.195]
Validation: 0it [00:00, ?it/s] 
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 167.17it/s, v_num=0, train_loss_step=0.725, train_loss_epoch=0.409, valid_loss=0.403]
Validation: 0it [00:00, ?it/s] 
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 143.06it/s, v_num=0, train_loss_step=0.292, train_loss_epoch=0.171, valid_loss=0.301]
(train_tune pid=701377) 
Validation: 0it [00:00, ?it/s] 
(train_tune pid=701377) 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(train_tune pid=701377) 
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 166.14it/s, v_num=0, train_loss_step=0.105, train_loss_epoch=0.0702, valid_loss=0.269] 
Validation: 0it [00:00, ?it/s] 
Epoch 599: 100%|██████████| 1/1 [00:00<00:00, 166.92it/s, v

(train_tune pid=701377) Global seed set to 8


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 162.89it/s, v_num=0, train_loss_step=12.20, train_loss_epoch=9.320]
Validation: 0it [00:00, ?it/s] 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 162.75it/s, v_num=0, train_loss_step=2.570, train_loss_epoch=4.130, valid_loss=6.400]
Validation: 0it [00:00, ?it/s] 
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 162.77it/s, v_num=0, train_loss_step=0.940, train_loss_epoch=0.888, valid_loss=2.540]
Validation: 0it [00:00, ?it/s] 
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 159.86it/s, v_num=0, train_loss_step=2.350, train_loss_epoch=0.750, valid_loss=0.137]
Validation: 0it [00:00, ?it/s] 
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 162.96it/s, v_num=0, train_loss_step=0.778, train_loss_epoch=1.630, valid_loss=0.173]
(train_tune pid=718828) 
Validation: 0it [00:00, ?it/s] 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(train_tune pid=718828) 
Epoch 599: 100%|██████████| 1/1 [00:00<00:00, 162.87it/s, v_num=0, train_loss_step=0.9

(train_tune pid=718828) Global seed set to 7


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 167.39it/s, v_num=0, train_loss_step=0.892, train_loss_epoch=0.692]
Validation: 0it [00:00, ?it/s] 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 163.50it/s, v_num=0, train_loss_step=0.599, train_loss_epoch=0.255, valid_loss=0.165]
Validation: 0it [00:00, ?it/s] 
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 166.50it/s, v_num=0, train_loss_step=0.189, train_loss_epoch=0.163, valid_loss=0.184]
(train_tune pid=718828) 
Validation: 0it [00:00, ?it/s] 
(train_tune pid=718828) 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(train_tune pid=718828) 
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 164.57it/s, v_num=0, train_loss_step=0.527, train_loss_epoch=0.264, valid_loss=0.219]
Validation: 0it [00:00, ?it/s] 
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 165.40it/s, v_num=0, train_loss_step=2.990, train_loss_epoch=0.409, valid_loss=0.165]
Validation: 0it [00:00, ?it/s] 
Epoch 599: 100%|██████████| 1/1 [00:00<00:00, 165.93it/s, v_n

(train_tune pid=718828) Global seed set to 17


Epoch 999: 100%|██████████| 1/1 [00:00<00:00, 164.84it/s, v_num=0, train_loss_step=0.0905, train_loss_epoch=0.299, valid_loss=0.178]
Validation: 0it [00:00, ?it/s] 
Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 160.55it/s, v_num=0, train_loss_step=0.705, train_loss_epoch=0.606]
Validation: 0it [00:00, ?it/s] 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 167.64it/s, v_num=0, train_loss_step=0.857, train_loss_epoch=0.899, valid_loss=0.320]
Validation: 0it [00:00, ?it/s] 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(train_tune pid=718828) 
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 166.90it/s, v_num=0, train_loss_step=0.686, train_loss_epoch=0.514, valid_loss=0.186]
Validation: 0it [00:00, ?it/s] 
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 166.90it/s, v_num=0, train_loss_step=0.452, train_loss_epoch=0.236, valid_loss=0.336]
Validation: 0it [00:00, ?it/s] 
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 168.00it/s, v_num=0, train_loss_step=0.615, train_loss_epoch=0.2